In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import mytools
import re

In [3]:
data_excel = pd.ExcelFile('G:/1-规划/崔开通崔可研规划/2020年任务清单/20200903/2020年任务清单（场景+MR）0903.xlsx')

In [4]:
data = data_excel.parse(sheet_name=data_excel.sheet_names[0])

In [5]:
data['TDL规划站号（新）'] = data['TDL规划站号（新）'].fillna('无')

In [6]:
qingdan_yuan = data.loc[data['分类']=='2020年5月新任务清单']

# -------全网进度--------
## 优化进度

In [7]:
qingdan = qingdan_yuan.copy()

In [8]:
# 优化总点youhua_z，优化完成youhua_w，
youhua_z = qingdan.loc[(qingdan['优化']=='优化')]['唯一编号'].count()
youhua_p = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]['唯一编号'].count()
youhua_w = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)& (qingdan['优化完成']==1)]['唯一编号'].count()

In [9]:
#youhua已经df，优化完成df
youhua_data_z = qingdan.loc[(qingdan['优化']=='优化') ]
youhua_data_p = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]
youhua_data_w = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1) & (qingdan['优化完成']==1)]

In [10]:
#地市的完成数量y_w，地市的派发总数量y_z
y_z = youhua_data_z.groupby(by = '地市').agg({'唯一编号':'count'})
y_w = youhua_data_w.groupby(by = '地市').agg({'唯一编号':'count'})
y_p = youhua_data_p.groupby(by = '地市').agg({'唯一编号':'count'})

In [11]:
# 统计每个地市的派发和完成
youhuajindu=pd.concat([y_z,y_p,y_w],axis = 1)
youhuajindu.columns = ['总量','派发','完成量']
youhuajindu['完成派发进度'] = youhuajindu['完成量']/youhuajindu['派发']
youhuajindu['整体进度'] = youhuajindu['完成量']/youhuajindu['总量']

In [12]:
#按照格式输出
youhua_wanchengdugao=list(youhuajindu.sort_values(by="完成派发进度",ascending= False).head(3).index)  #排序 升序True 降序False by列

In [13]:
'MR竞对优化工单总计'+str(youhua_z)+'单。已派发'+str(youhua_p)+'已完成'+str(youhua_w)+ '解决率'+str(youhua_w/youhua_p)+ \
str(youhua_wanchengdugao[0])+'、'+str(youhua_wanchengdugao[1])+'、'+str(youhua_wanchengdugao[2])+'派发工单解决率相对较高'

'MR竞对优化工单总计4313单。已派发1451已完成947解决率0.6526533425223984随州、襄阳、荆州派发工单解决率相对较高'

In [14]:
youhuajindu.sort_values(by="完成派发进度",ascending= False)

,总量,派发,完成量,完成派发进度,整体进度
地市,,,,,
随州,115,59,49,0.830508,0.426087
襄阳,217,83,67,0.807229,0.308756
荆州,197,129,103,0.798450,0.522843
恩施,633,177,125,0.706215,0.197472
孝感,302,101,70,0.693069,0.231788
武汉,1077,315,218,0.692063,0.202414
天门,42,42,29,0.690476,0.690476
荆门,165,66,44,0.666667,0.266667
潜江,34,28,18,0.642857,0.529412


# 催开进度

In [15]:
qingdan = qingdan_yuan.copy()

In [16]:
# cuikai的数量统计
cuikai_zdanzi = qingdan.loc[(qingdan['催开']=='催开') ]['唯一编号'].count()
cuikai_z = qingdan.loc[(qingdan['催开']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()
cuikai_w = qingdan.loc[(
    qingdan['催开']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [17]:
#催开总量和完成的df
cuikai_quchong = qingdan.loc[(qingdan['催开']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') 
cuikai_quchong_wancheng = qingdan.loc[(
    qingdan['催开']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') 

In [18]:
#个地市的进度
cuikai_zongliang = cuikai_quchong.groupby(by = '地市').agg({'唯一编号':'count'})
cuikai_wanchengliang = cuikai_quchong_wancheng.groupby(by = '地市').agg({'唯一编号':'count'})
cuikaijindu=pd.concat([cuikai_zongliang,cuikai_wanchengliang],axis = 1)
cuikaijindu.columns = ['总量','完成量']
cuikaijindu['进度'] = cuikaijindu['完成量']/cuikaijindu['总量']

In [19]:
cuikai_wanchengdudi=list(cuikaijindu.sort_values(by="进度",ascending= True).head(1).index)  #排序 升序True 降序False by列
a=cuikai_wanchengdudi[0]
jindudi=cuikai_quchong.loc[(cuikai_quchong['地市']==a) & (cuikai_quchong['催开完成']!=1)
                          ].groupby(by='工期核实').agg({'唯一编号':'count'}).sort_values(by="唯一编号",ascending= False).head(1).index
'催开工单共'+str(cuikai_zdanzi)+'单，涉及逻辑站\
点'+str(cuikai_z)+'个，已开通站点'+str(cuikai_w)+'个(本月新增201个），催开完成率\
为'+str(cuikai_w/cuikai_z*100)[:5]+'%'+'。'+cuikai_wanchengdudi[0]+'地市整体解决进度较慢，主要为'+jindudi[0]+'未开通占比较\
高，需加快站点建设进度。'+'已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

'催开工单共2827单，涉及逻辑站点1760个，已开通站点867个(本月新增201个），催开完成率为49.26%。荆门地市整体解决进度较慢，主要为5月前完成-2020年湖北无线网4G主设备扩容工程第一批未开通占比较高，需加快站点建设进度。已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

In [20]:
cuikaijindu.sort_values(by="进度",ascending= True)

,总量,完成量,进度
地市,,,
荆门,55,12,0.218182
天门,27,6,0.222222
潜江,8,2,0.250000
咸宁,170,48,0.282353
恩施,129,39,0.302326
鄂州,18,6,0.333333
黄石,97,36,0.371134
荆州,101,41,0.405941
随州,53,23,0.433962


In [21]:
cuikai_quchong.to_clipboard()

# 可研进度

In [22]:
qingdan = qingdan_yuan.copy()

In [23]:
keyan_zdanzi = qingdan.loc[(qingdan['催可研']=='可研') ]['唯一编号'].count()
keyan_z = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()
keyan_w = qingdan.loc[(
    qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()

In [24]:
'催可研工单共'+str(keyan_zdanzi)+'单，涉及'+str(keyan_z)+'个逻辑站点工期催研，目前累\
计'+str(keyan_w)+'个站点已纳入工期，入研进度为'+str(keyan_w/keyan_z*100)[:5]+'%。'

'催可研工单共2029单，涉及1243个逻辑站点工期催研，目前累计784个站点已纳入工期，入研进度为63.07%。'

In [25]:
#可研总量和完成的df
keyan_quchong = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）') 
keyan_quchong_wancheng = qingdan.loc[(
    qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(subset='TDL规划站号（新）') 

In [26]:
#个地市的进度
keyan_zongliang = keyan_quchong.groupby(by = '地市').agg({'唯一编号':'count'})
keyan_wanchengliang = keyan_quchong_wancheng.groupby(by = '地市').agg({'唯一编号':'count'})
keyanjindu=pd.concat([keyan_zongliang,keyan_wanchengliang],axis = 1)
keyanjindu.columns = ['总量','完成量']
keyanjindu['进度'] = keyanjindu['完成量']/keyanjindu['总量']

In [27]:
keyanjindu

,总量,完成量,进度
地市,,,
十堰,104,47,0.451923
咸宁,200,127,0.635000
天门,27,22,0.814815
孝感,85,55,0.647059
宜昌,75,14,0.186667
恩施,102,79,0.774510
武汉,111,101,0.909910
江汉,18,15,0.833333
潜江,6,3,0.500000


# 规划进度

In [28]:
qingdan = qingdan_yuan.copy()

In [29]:
guihua_zdanzi = qingdan.loc[(qingdan['规划']=='规划') ]['唯一编号'].count()
guihua_w = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1) ]['唯一编号'].count()
'需规划解决工单共'+str(guihua_zdanzi)+'单，目前累计完成'+str(guihua_w)+'个问题工单规划衔接，规\
划完成比例'+str(guihua_w/guihua_zdanzi*100)[:5]+'%。'

'需规划解决工单共6492单，目前累计完成2604个问题工单规划衔接，规划完成比例40.11%。'

In [30]:
#规划总量和完成的df
guihua_quchong = qingdan.loc[(qingdan['规划']=='规划') ]
guihua_quchong_wancheng = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1) ]

In [31]:
#个地市的进度
guihua_zongliang = guihua_quchong.groupby(by = '地市').agg({'唯一编号':'count'})
guihua_wanchengliang = guihua_quchong_wancheng.groupby(by = '地市').agg({'唯一编号':'count'})
guihuajindu=pd.concat([guihua_zongliang,guihua_wanchengliang],axis = 1)
guihuajindu.columns = ['总量','完成量']
guihuajindu['进度'] = guihuajindu['完成量']/guihuajindu['总量']

# ---------周进度----------

In [36]:
line = '''
1.针对2020年的MR竞对覆盖提升清单，总任务量共计12009个。
 规（小区）/优（小区）/研（站点）/建（站点）分别为6492/ 4313/ 1245/1716,
 本周新增分别为0/7/0/11，累计已完成2510（38.66%）/ 944（21.89%）/ 763（61.29%）/ 838（48.83%）。
 '''

In [37]:
keyanjindu.columns = ['总量',  '完成量', '可研进度']
keyanjindu1=keyanjindu[['可研进度']]
cuikaijindu.columns = ['总量',  '完成量', '催开进度']
cuikaijindu1=cuikaijindu[['催开进度']]
youhuajindu.columns = ['总量', '派发量', '完成量','完成派发进度', '优化进度']
youhuajindu1=youhuajindu[['优化进度']]
guihuajindu.columns = ['总量', '完成量', '规划进度']
guihuajindu1=guihuajindu[['规划进度']]
data_z = pd.concat([youhuajindu1,cuikaijindu1,keyanjindu1,guihuajindu1],axis=1)

In [38]:
pattern = re.compile(r'.*累计已完成(\d*).*/ (\d*).*/ (\d*).*/ (\d*).*')                    # 用于匹配至少一个数字
m = pattern.search(line) 
g_s = m.group(1)
y_s = m.group(2)
k_s = m.group(3)
j_s = m.group(4)
gui = guihua_w-int(g_s)
you = youhua_w-int(y_s)
yan = keyan_w-int(k_s)
jian = cuikai_w-int(j_s)

In [39]:
gui,you,yan,jian

(94, 3, 21, 29)

In [40]:
print('1.针对2020年的MR竞对覆盖提升清单，总任务量共计12009个。\n \
规（小区）/优（小区）/研（站点）/建（站点）分别为{g}/ {y}/ {k}/{j},\n \
本周新增分别为{gui}/{you}/{yan}/{jian}，累计已完成{g_w}（{gui_b}%）/ {y_w}（{you_b}%）/ {ky_w}（{yan_b}%）/ {c_w}（{jian_b}%）。'.format(
    g_w=str(guihua_w),ky_w=str(keyan_w),c_w =str(cuikai_w),y_w = str(youhua_w) ,g=str(guihua_zdanzi),
    y=str(youhua_z),k=str(keyan_z),j=str(cuikai_z),gui=gui  ,you= you ,yan= yan ,jian = jian ,
    gui_b=round(guihua_w/guihua_zdanzi*100,2),
    you_b=round(youhua_w/youhua_z*100,2),
    yan_b=round(keyan_w/keyan_z*100,2),
    jian_b=round(cuikai_w/cuikai_z*100,2)
))

1.针对2020年的MR竞对覆盖提升清单，总任务量共计12009个。
 规（小区）/优（小区）/研（站点）/建（站点）分别为6492/ 4313/ 1243/1760,
 本周新增分别为94/3/21/29，累计已完成2604（40.11%）/ 947（21.96%）/ 784（63.07%）/ 867（49.26%）。


# ------------武汉竞对及MR分析----------------
### 优化

In [ ]:
qingdan_y = qingdan_yuan.copy()
qingdan_y = qingdan_y.loc[qingdan_y['地市']=='武汉']
qingdan = qingdan_y.loc[qingdan_y['地市']=='武汉']

In [ ]:
qingdan.columns

In [ ]:
youhua_z = qingdan.loc[(qingdan['优化']=='优化')]['唯一编号'].count()
youhua_p = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]['唯一编号'].count()
youhua_w = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)& (qingdan['优化完成']==1)]['唯一编号'].count()

In [ ]:
youhua_w

In [ ]:
youhua_data_z = qingdan.loc[(qingdan['优化']=='优化') ]
youhua_data_p = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1)]
youhua_data_w = qingdan.loc[(qingdan['优化']=='优化') & (qingdan['优化已派发']==1) & (qingdan['优化完成']==1)]

In [ ]:
youhua_data_p.shape

In [ ]:
y_w = youhua_data_w.groupby(by = '区县').agg({'唯一编号':'count'})
y_p = youhua_data_p.groupby(by = '区县').agg({'唯一编号':'count'})
y_z = youhua_data_z.groupby(by = '区县').agg({'唯一编号':'count'})
youhuajindu=pd.concat([y_z,y_p,y_w],axis = 1)

In [ ]:
youhuajindu.columns = ['总量','派发量','完成量']
youhuajindu['派发完成进度'] = youhuajindu['完成量']/youhuajindu['派发量']
youhuajindu['进度'] = youhuajindu['完成量']/youhuajindu['总量']

In [ ]:
youhuajindu

In [ ]:
youhua_wanchengdugao=list(youhuajindu.sort_values(by="派发完成进度",ascending= False).head(3).index)  #排序 升序True 降序False by列

In [ ]:
'MR竞对优化工单'+str(youhua_z)+'单。已派发'+str(youhua_p)+'已完成'+str(youhua_w)+'完成率为'+str(youhua_w/youhua_p*100)[:5]+'%'+str(
    youhua_wanchengdugao[0])+'，'+str(youhua_wanchengdugao[1])+'，'+str(youhua_wanchengdugao[2])+'派发工单解决率相对较高'

In [ ]:
youhuajindu.to_clipboard()

# 催开

In [ ]:
qingdan_c = qingdan_yuan.copy()
qingdan_c = qingdan_c.loc[qingdan_c['地市']=='武汉']
qingdan = qingdan_c.copy()

In [ ]:
cuikai_zdanzi = qingdan.loc[(qingdan['催开']=='催开') ]['唯一编号'].count()
cuikai_z = qingdan.loc[(qingdan['催开']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()
cuikai_w = qingdan.loc[(qingdan['催开']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(
    subset='TDL规划站号（新）') ['唯一编号'].count()

In [ ]:
cuikai_quchong = qingdan.loc[(qingdan['催开']=='催开') ].drop_duplicates(subset='TDL规划站号（新）') 
cuikai_quchong_wancheng = qingdan.loc[(qingdan['催开']=='催开') & (qingdan['催开完成']==1) ].drop_duplicates(
    subset='TDL规划站号（新）') 

In [ ]:
cuikai_zongliang = cuikai_quchong.groupby(by = '区县').agg({'唯一编号':'count'})
cuikai_wanchengliang = cuikai_quchong_wancheng.groupby(by = '区县').agg({'唯一编号':'count'})

In [ ]:
cuikaijindu=pd.concat([cuikai_zongliang,cuikai_wanchengliang],axis = 1)
cuikaijindu.columns = ['总量','完成量']
cuikaijindu['进度'] = cuikaijindu['完成量']/cuikaijindu['总量']

In [ ]:
cuikai_wanchengdudi=list(cuikaijindu.sort_values(by="进度",ascending= True).head(1).index)  #排序 升序True 降序False by列

a=cuikai_wanchengdudi[0]

jindudi=cuikai_quchong.loc[(cuikai_quchong['区县']==a) & (cuikai_quchong['催开完成']!=1)].groupby(by='工期核实').agg(
    {'唯一编号':'count'}).sort_values(by="唯一编号",ascending= False).head(1).index

In [ ]:
a

In [ ]:
'催开工单共'+str(cuikai_zdanzi)+'单，涉及逻辑站\
点'+str(cuikai_z)+'个，已开通站点'+str(cuikai_w)+'个(本月新增201个），催开完成率\
为'+str(cuikai_w/cuikai_z*100)[:5]+'%'+'。'+cuikai_wanchengdudi[0]+'地市整体解决进度较慢，主要为'+jindudi[0]+'未开通占比较\
高，需加快站点建设进度。'+'已催开工单整体MR覆盖率改善2.60pp，对全网劣于竞对小区占比指标改善5.26pp。'

In [ ]:
cuikaijindu

In [ ]:
cuikaijindu.to_clipboard()

# 可研

In [ ]:
qingdan_k = qingdan_yuan.copy()
qingdan_k = qingdan_k.loc[qingdan_k['地市']=='武汉']
qingdan = qingdan_k.copy()

In [ ]:
keyan_zdanzi = qingdan.loc[(qingdan['催可研']=='可研') ]['唯一编号'].count()
keyan_z = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）') ['唯一编号'].count()
keyan_w = qingdan.loc[(qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(
    subset='TDL规划站号（新）') ['唯一编号'].count()

In [ ]:
'催可研工单共'+str(keyan_zdanzi)+'单，涉及'+str(keyan_z)+'个逻辑站点工期催研，目前累\
计'+str(keyan_w)+'个站点已纳入工期，入研进度为'+str(keyan_w/keyan_z*100)[:5]+'%。'

In [ ]:
keyan_quchong = qingdan.loc[(qingdan['催可研']=='可研') ].drop_duplicates(subset='TDL规划站号（新）')
keyan_quchong_w = qingdan.loc[(qingdan['催可研']=='可研') & (qingdan['可研完成']==1) ].drop_duplicates(subset='TDL规划站号（新）')

In [ ]:
k_z = keyan_quchong.groupby(by = '区县').agg({'唯一编号':'count'})
k_w = keyan_quchong_w.groupby(by = '区县').agg({'唯一编号':'count'})
keyanjindu=pd.concat([k_z,k_w],axis = 1)
keyanjindu.columns = ['总量','完成量']
keyanjindu['进度'] = keyanjindu['完成量']/keyanjindu['总量']

In [ ]:
keyanjindu.to_clipboard()

# 规划 

In [ ]:
qingdan_g = qingdan_yuan.copy()
qingdan_g = qingdan_g.loc[qingdan_g['地市']=='武汉']
qingdan = qingdan_g.copy()

guihua_zdanzi = qingdan.loc[(qingdan['规划']=='规划') ]['唯一编号'].count()
guihua_w = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1) ]['唯一编号'].count()

In [ ]:
'需规划解决工单共'+str(guihua_zdanzi)+'单，目前累计完成'+str(guihua_w)+'个问题工单规划衔接，规\
划完成比例'+str(guihua_w/guihua_zdanzi*100)[:5]+'%。'

In [ ]:
guihua_qingdan = qingdan.loc[(qingdan['规划']=='规划') ]
guihua_qingdan_w = qingdan.loc[(qingdan['规划']=='规划') & (qingdan['规划完成']==1)]
g_z = guihua_qingdan.groupby(by = '区县').agg({'唯一编号':'count'})
g_w = guihua_qingdan_w.groupby(by = '区县').agg({'唯一编号':'count'})
guihuajindu=pd.concat([g_z,g_w],axis = 1)
guihuajindu.columns = ['总量','完成量']
guihuajindu['进度'] = guihuajindu['完成量']/guihuajindu['总量']

In [ ]:
guihuajindu.to_clipboard()